In [1]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import time

In [2]:
# 1. 데이터 생성 (10,000개의 128차원 벡터)
d = 128 
nb = 10000  
nq = 5  

np.random.seed(42)
xb = np.random.random((nb, d)).astype('float32') # DB용 데이터
xq = np.random.random((nq, d)).astype('float32') # 검색용 쿼리 데이터

print(f"데이터셋 준비 완료: {nb}개의 벡터 (차원: {d})")

데이터셋 준비 완료: 10000개의 벡터 (차원: 128)


# Flat Index

In [3]:
index_flat = faiss.IndexFlatL2(d)   # L2 거리 기준 전수 조사 인덱스
index_flat.add(xb)                  # 데이터 추가

In [4]:
start_time = time.time()
D_flat, I_flat = index_flat.search(xq, 5) # 상위 5개 검색
flat_time = time.time() - start_time

print(f"\n[Flat Index] 검색 완료 | 소요 시간: {flat_time:.6f}초")
print(f"가장 가까운 데이터 인덱스: {I_flat[0]}")


[Flat Index] 검색 완료 | 소요 시간: 0.006076초
가장 가까운 데이터 인덱스: [8769 9385   82 5125 9571]


# IVF Index

In [5]:
nlist = 100 # 클러스터 개수
quantizer = faiss.IndexFlatL2(d)
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist)

In [6]:
# Train
index_ivf.train(xb)
index_ivf.add(xb)
index_ivf.nprobe = 10  # 검색 시 몇 개의 클러스터를 확인할지 설정 (높을수록 정확)

In [7]:
start_time = time.time()
D_ivf, I_ivf = index_ivf.search(xq, 5)
ivf_time = time.time() - start_time

print(f"[IVF Index]  검색 완료 | 소요 시간: {ivf_time:.6f}초")
print(f"가장 가까운 데이터 인덱스: {I_ivf[0]}")

[IVF Index]  검색 완료 | 소요 시간: 0.001053초
가장 가까운 데이터 인덱스: [8769 9571 3948 4436 1056]


# HNSW Index

In [8]:
# M: 각 점이 연결될 이웃의 수 (보통 16~32)
index_hnsw = faiss.IndexHNSWFlat(d, 32)
index_hnsw.add(xb)

In [9]:
start_time = time.time()
D_hnsw, I_hnsw = index_hnsw.search(xq, 5)
hnsw_time = time.time() - start_time

print(f"[HNSW Index] 검색 완료 | 소요 시간: {hnsw_time:.6f}초")
print(f"가장 가까운 데이터 인덱스: {I_hnsw[0]}")

[HNSW Index] 검색 완료 | 소요 시간: 0.000894초
가장 가까운 데이터 인덱스: [8769 9385 5125 9571 3491]


In [10]:
print(f"Flat 대비 IVF  속도 향상: {flat_time / ivf_time:.2f}배")
print(f"Flat 대비 HNSW 속도 향상: {flat_time / hnsw_time:.2f}배")

Flat 대비 IVF  속도 향상: 5.77배
Flat 대비 HNSW 속도 향상: 6.80배


# PQ Index

In [11]:
m = 8      # 벡터를 8개의 서브 벡터로 쪼갬 (차원 d가 m의 배수여야 함)
nbits = 8  # 각 서브 벡터를 8비트로 압축 (256개의 중심점 사용)

In [12]:
# Train 
index_pq = faiss.IndexPQ(d, m, nbits)

index_pq.train(xb)
index_pq.add(xb)

In [13]:
start_time = time.time()
D_pq, I_pq = index_pq.search(xq, 5)
pq_time = time.time() - start_time

print(f"[PQ Index]   검색 완료 | 소요 시간: {pq_time:.6f}초")
print(f"가장 가까운 데이터 인덱스: {I_pq[0]}")

[PQ Index]   검색 완료 | 소요 시간: 0.001640초
가장 가까운 데이터 인덱스: [8769 9571 4342 1983 7375]


In [14]:
print(f"Flat 대비 IVF  속도 향상: {flat_time / ivf_time:.2f}배")
print(f"Flat 대비 HNSW 속도 향상: {flat_time / hnsw_time:.2f}배")
print(f"Flat 대비 PQ   속도 향상: {flat_time / pq_time:.2f}배")

Flat 대비 IVF  속도 향상: 5.77배
Flat 대비 HNSW 속도 향상: 6.80배
Flat 대비 PQ   속도 향상: 3.71배


# Advanced

In [15]:
file_path = 'sample_sentences.txt'
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        sentences = [line.strip() for line in f.readlines() if line.strip()]
    print(f"총 {len(sentences)}개의 문장을 성공적으로 불러왔습니다.")

except FileNotFoundError:
    print("에러: 파일을 찾을 수 없습니다.")
    sentences = []

총 50개의 문장을 성공적으로 불러왔습니다.


In [16]:
# 임베딩 모델 
model = SentenceTransformer('all-MiniLM-L6-v2')

In [17]:
# 임베딩 
sentence_embeddings = model.encode(sentences).astype('float32')

In [25]:
# 각 인덱스 초기화 및 데이터 추가
d = sentence_embeddings.shape[1]

# (1) Flat Index
idx_flat = faiss.IndexFlatL2(d)
idx_flat.add(sentence_embeddings)

# (2) IVF Index
nlist = 5
quantizer = faiss.IndexFlatL2(d)
idx_ivf = faiss.IndexIVFFlat(quantizer, d, nlist)
idx_ivf.cp.min_points_per_centroid = 1  # 소량 데이터라 하한선을 강제로 낮춤 
idx_ivf.train(sentence_embeddings)
idx_ivf.add(sentence_embeddings)
idx_ivf.nprobe = 2

# (3) HNSW Index
idx_hnsw = faiss.IndexHNSWFlat(d, 32)
idx_hnsw.add(sentence_embeddings)

# (4) PQ Index
m = 8
nbits = 4   # 데이터가 50개, 2^4 = 16
idx_pq = faiss.IndexPQ(d, m, nbits)
idx_pq.train(sentence_embeddings)
idx_pq.add(sentence_embeddings)

WARNING clustering 50 points to 16 centroids: please provide at least 624 training points
WARNING clustering 50 points to 16 centroids: please provide at least 624 training points
WARNING clustering 50 points to 16 centroids: please provide at least 624 training points
WARNING clustering 50 points to 16 centroids: please provide at least 624 training points
WARNING clustering 50 points to 16 centroids: please provide at least 624 training points
WARNING clustering 50 points to 16 centroids: please provide at least 624 training points
WARNING clustering 50 points to 16 centroids: please provide at least 624 training points
WARNING clustering 50 points to 16 centroids: please provide at least 624 training points


In [26]:
# 2. User Query (for Test)
query_text = "I want to know the weather for tomorrow."
query_vector = model.encode([query_text]).astype('float32')

In [27]:
# 3. 각 인덱스에서 검색 수행 (Top 3)
k = 3
D_f, I_f = idx_flat.search(query_vector, k)
D_ivf, I_ivf = idx_ivf.search(query_vector, k)
D_h, I_h = idx_hnsw.search(query_vector, k)
D_h, I_h = idx_pq.search(query_vector, k)

In [28]:
# 검색 대상 인덱스 
index_dict = {
    "Flat Index (Exact)": idx_flat,
    "IVF Index": idx_ivf,
    "HNSW Index": idx_hnsw,
    "PQ Index": idx_pq
}

In [ ]:
# 검색 결과 확인 
# 1. 기준이 되는 Flat 검색 결과 먼저 수행
D_f, I_f = idx_flat.search(query_vector, k)

# 2. 각 인덱스별 순회하며 결과 출력
for name, index in index_dict.items():
    start_time = time.time()
    D, I = index.search(query_vector, k)
    duration = time.time() - start_time
    
    print(f"[{name}] 검색 완료 | 소요 시간: {duration:.6f}s")
    
    for i in range(k):
        # Flat Index의 결과와 비교하여 일치 여부 확인
        is_correct = "일치" if I[0][i] == I_f[0][i] else "불일치"
        print(f"  {i+1}위: {sentences[I[0][i]]} (결과: {is_correct})")
    
    print("-" * 50)

[Flat Index (Exact)] 검색 완료 | 소요 시간: 0.000023s
  1위: The weather forecast predicts a snowy winter. (결과: 일치)
  2위: The desert is extremely hot during the day. (결과: 일치)
  3위: The sun rises in the east every morning. (결과: 일치)
--------------------------------------------------
[IVF Index] 검색 완료 | 소요 시간: 0.000500s
  1위: The weather forecast predicts a snowy winter. (결과: 일치)
  2위: The desert is extremely hot during the day. (결과: 일치)
  3위: The sun rises in the east every morning. (결과: 일치)
--------------------------------------------------
[HNSW Index] 검색 완료 | 소요 시간: 0.000200s
  1위: The weather forecast predicts a snowy winter. (결과: 일치)
  2위: The desert is extremely hot during the day. (결과: 일치)
  3위: The sun rises in the east every morning. (결과: 일치)
--------------------------------------------------
[PQ Index] 검색 완료 | 소요 시간: 0.001047s
  1위: Heavy rain is expected in the coastal areas. (결과: 불일치)
  2위: The desert is extremely hot during the day. (결과: 일치)
  3위: The weather forecast predicts a snow

In [30]:
# 검색 결과 확인 (거리 비교)
# 1. 기준이 되는 Flat 검색 결과 수행
D_f, I_f = idx_flat.search(query_vector, k)

# 2. 각 인덱스별 순회하며 결과 출력
for name, index in index_dict.items():
    start_time = time.time()
    # D: 거리값(Distance), I: 인덱스 번호(Index)
    D, I = index.search(query_vector, k) 
    duration = time.time() - start_time
    
    print(f"[{name}] 검색 완료 | 소요 시간: {duration:.6f}s")
    
    for i in range(k):
        # Flat Index의 결과와 비교하여 일치 여부 확인
        is_correct = "일치" if I[0][i] == I_f[0][i] else "불일치"
        # D[0][i]가 바로 해당 결과와의 거리값입니다.
        distance_val = D[0][i]
        
        print(f"  {i+1}위: {sentences[I[0][i]][:40]}... ")
        print(f"       [상태: {is_correct} | 거리: {distance_val:.4f}]")
    
    print("-" * 60)

[Flat Index (Exact)] 검색 완료 | 소요 시간: 0.000171s
  1위: The weather forecast predicts a snowy wi... 
       [상태: 일치 | 거리: 1.0320]
  2위: The desert is extremely hot during the d... 
       [상태: 일치 | 거리: 1.3093]
  3위: The sun rises in the east every morning.... 
       [상태: 일치 | 거리: 1.3235]
------------------------------------------------------------
[IVF Index] 검색 완료 | 소요 시간: 0.000172s
  1위: The weather forecast predicts a snowy wi... 
       [상태: 일치 | 거리: 1.0320]
  2위: The desert is extremely hot during the d... 
       [상태: 일치 | 거리: 1.3093]
  3위: The sun rises in the east every morning.... 
       [상태: 일치 | 거리: 1.3235]
------------------------------------------------------------
[HNSW Index] 검색 완료 | 소요 시간: 0.000129s
  1위: The weather forecast predicts a snowy wi... 
       [상태: 일치 | 거리: 1.0320]
  2위: The desert is extremely hot during the d... 
       [상태: 일치 | 거리: 1.3093]
  3위: The sun rises in the east every morning.... 
       [상태: 일치 | 거리: 1.3235]
-------------------------------------

---
End of Documents